In [1]:
import sys
sys.path.append('..')
import pandas as pd
%load_ext autoreload
%autoreload 2
import spacy
import ru2
from IPython.display import display

In [2]:
pbool = lambda x: '+' if x else '-'
def pipe_add(n, names=['sbd']):
    pos = 0
    for x in names:
        if not x in n.pipe_names:
            if pos >= len(n.pipe_names):
                n.add_pipe(n.create_pipe(x))
                pos += 1
            else:
                n.add_pipe(n.create_pipe(x), before=n.pipe_names[pos])
                pos += 1
    return n

nlp_ru=pipe_add(spacy.load('../ru2'), ['sbd', 'tagger', 'parser'])
print("RU pipeline: {}".format(nlp_ru.pipe_names))

RU pipeline: ['sbd', 'tagger', 'parser', 'ner']


In [3]:
def parse(nlp, text):
    pbool = lambda x: '+' if x else '-'
    doc = nlp(text)
    sents=pd.DataFrame(data=[(str(s),) for s in doc.sents], columns=['Sentence']);
    print('NLP={}.{} Text={}'.format(nlp.__class__.__module__, nlp.__class__.__name__, doc))
    display(sents.head())
    toks=pd.DataFrame(data=[(t.shape_, pbool(t.orth_ in nlp.vocab), t.pos_,
                             t.text, t.tag_, t.lemma_, t.dep_, t.head) for t in doc],
                     columns=['shape', 'vocab', 'POS', 'text', 'tag', 'lemma', 'dep', 'head']);
    display(toks)
    #from spacy import displacy
    #displacy.serve(doc, style='dep')
text = 'Она дочь крокодила из соседней галактики . Восславим дочь крокодила ! ' # + 'Карамамба !' 
#text = 'Налоги пропитаны потом всякого, кто трудится .'
parse(nlp_ru, text)


NLP=spacy.lang.ru.Russian Text=Она дочь крокодила из соседней галактики . Восславим дочь крокодила ! 


,Sentence
0,Она дочь крокодила из соседней галактики .
1,Восславим дочь крокодила !


,shape,vocab,POS,text,tag,lemma,dep,head
0,Xxx,+,PRON,Она,PRON__Case=Nom|Gender=Fem|Number=Sing|Person=3,она,nsubj,крокодила
1,xxxx,+,NOUN,дочь,NOUN__Animacy=Anim|Case=Acc|Gender=Fem|Number=...,дочь,obj,крокодила
2,xxxx,-,VERB,крокодила,VERB__Aspect=Imp|Gender=Fem|Mood=Ind|Number=Si...,крокодила,ROOT,крокодила
3,xx,+,ADP,из,ADP___,из,case,галактики
4,xxxx,+,ADJ,соседней,ADJ__Case=Gen|Degree=Pos|Gender=Fem|Number=Sing,соседний,amod,галактики
5,xxxx,+,NOUN,галактики,NOUN__Animacy=Inan|Case=Gen|Gender=Fem|Number=...,галактика,obl,крокодила
6,.,+,PUNCT,.,PUNCT___,.,punct,крокодила
7,Xxxxx,-,VERB,Восславим,VERB__Aspect=Perf|Mood=Ind|Number=Plur|Person=...,восславить,nsubj,крокодила
8,xxxx,+,NOUN,дочь,NOUN__Animacy=Anim|Case=Acc|Gender=Fem|Number=...,дочь,flat:name,Восславим
9,xxxx,-,VERB,крокодила,VERB__Aspect=Imp|Gender=Fem|Mood=Ind|Number=Si...,крокодила,conj,крокодила


In [4]:
ru2.lemmatizer.DEBUG = 1
nlp_r2=pipe_add(ru2.Russian2())
print("R2 pipeline: {}".format(nlp_r2.pipe_names))
parse(nlp_r2, text)

R2 pipeline: ['sbd']
NLP=ru2.Russian2 Text=Она дочь крокодила из соседней галактики . Восславим дочь крокодила ! 


,Sentence
0,Она дочь крокодила из соседней галактики .
1,Восславим дочь крокодила !


No tagger, so we do Lemmatizer.lookup: Она
No tagger, so we do Lemmatizer.lookup: дочь
No tagger, so we do Lemmatizer.lookup: крокодила
No tagger, so we do Lemmatizer.lookup: из
Found 2 lookup alternatives for: из
No tagger, so we do Lemmatizer.lookup: соседней
No tagger, so we do Lemmatizer.lookup: галактики
No tagger, so we do Lemmatizer.lookup: .
No tagger, so we do Lemmatizer.lookup: Восславим
No tagger, so we do Lemmatizer.lookup: дочь
No tagger, so we do Lemmatizer.lookup: крокодила
No tagger, so we do Lemmatizer.lookup: !


,shape,vocab,POS,text,tag,lemma,dep,head
0,Xxx,+,,Она,,она,,Она
1,xxxx,+,,дочь,,дочь,,дочь
2,xxxx,+,,крокодила,,крокодил,,крокодила
3,xx,+,,из,,из,,из
4,xxxx,+,,соседней,,соседний,,соседней
5,xxxx,+,,галактики,,галактика,,галактики
6,.,+,,.,,.,,.
7,Xxxxx,+,,Восславим,,восславить,,Восславим
8,xxxx,+,,дочь,,дочь,,дочь
9,xxxx,+,,крокодила,,крокодил,,крокодила


In [5]:
nlp_en=pipe_add(spacy.load('en'))
print("EN pipeline: {}".format(nlp_en.pipe_names))
parse(nlp_en, text)
nlp_xx=pipe_add(spacy.load('xx'))
print("XX pipeline: {}".format(nlp_xx.pipe_names))
parse(nlp_xx, text)

EN pipeline: ['sbd', 'tagger', 'parser', 'ner']
NLP=spacy.lang.en.English Text=Она дочь крокодила из соседней галактики . Восславим дочь крокодила ! 


,Sentence
0,Она дочь крокодила из соседней галактики .
1,Восславим дочь крокодила !


,shape,vocab,POS,text,tag,lemma,dep,head
0,Xxx,-,PROPN,Она,NNP,она,amod,крокодила
1,xxxx,-,VERB,дочь,VBP,дочь,compound,крокодила
2,xxxx,-,NOUN,крокодила,NN,крокодила,nsubj,из
3,xx,+,ADJ,из,JJ,из,ROOT,из
4,xxxx,-,NOUN,соседней,NNS,соседней,amod,галактики
5,xxxx,-,VERB,галактики,VBN,галактики,dobj,из
6,.,+,PUNCT,.,.,.,punct,из
7,Xxxxx,-,NOUN,Восславим,NN,восславим,compound,крокодила
8,xxxx,-,NOUN,дочь,NN,дочь,compound,крокодила
9,xxxx,-,NOUN,крокодила,NN,крокодила,ROOT,крокодила


XX pipeline: ['sbd', 'ner']
NLP=spacy.lang.xx.MultiLanguage Text=Она дочь крокодила из соседней галактики . Восславим дочь крокодила ! 


,Sentence
0,Она дочь крокодила из соседней галактики .
1,Восславим дочь крокодила !


,shape,vocab,POS,text,tag,lemma,dep,head
0,Xxx,-,,Она,,Она,,Она
1,xxxx,-,,дочь,,дочь,,дочь
2,xxxx,-,,крокодила,,крокодила,,крокодила
3,xx,+,,из,,из,,из
4,xxxx,-,,соседней,,соседней,,соседней
5,xxxx,-,,галактики,,галактики,,галактики
6,.,+,,.,,.,,.
7,Xxxxx,-,,Восславим,,Восславим,,Восславим
8,xxxx,-,,дочь,,дочь,,дочь
9,xxxx,-,,крокодила,,крокодила,,крокодила


In [6]:
def inspect_object(x):
    import inspect
    opts = dir(x)
    
    for p in opts:
        if not '__' in p and not (p+'_' in opts) and not p.startswith('is_') and not p.startswith('like_'):
            try:
                a = getattr(x, p)
            except Exception as e:
                continue
                a = 'Exception: {}'.format(e)
            if callable(a): # don't call callables to avoid the object state change
                #print('.{}()'.format(a))
                continue
#                 try:
#                     #a = a()
#                     p = p + '()'
#                 except:
#                     pass
            if inspect.isgenerator(a):
                p = p+'[:]'
                a = list(a)
            #if hasattr(a, '__iter__'): a = list(a)
            print('.{} = {}'.format(p, a))

In [7]:
inspect_object(nlp_ru.vocab['она'])

.cluster = 0
.flags = 12306
.has_vector = False
.lang_ = ru
.lower_ = она
.norm_ = она
.orth_ = она
.prefix_ = о
.prob = -20.0
.rank = 0
.sentiment = 0.0
.shape_ = xxx
.suffix_ = она
.text = она
.vocab = <spacy.vocab.Vocab object at 0x7f442e102548>


In [8]:
vocab = sorted([n.orth_ for n in nlp_ru.vocab])
print(len(vocab), ':', ' '.join(vocab[::400]))

108205 : 	 1759 22,28 6,6 Bork LadyCare VOLVO Аваль Аналогичных БДО Беслан Буквы Ведомство Во-вторых Вузу Гвардейской Госдумой Дармштадт Добраться Единства Закончил Ивановича Ирвингов Камчатку Климент Кору Лаар Лоренцо Манфред Милане Муашшар Насера Нивы Оборот Органы ПАЛ Перекукует Подготовлены Португалия Приезжая Прямой Рао Рокот Сайнс Сены Соберет Средиземном Счастливый Тирита УФСБ Уфе Фрейда ЦЕН Чуровым Экспертного аВт автофургона актуальные анонимного армии афганцев баталии безупречную бессмысленности благоденствует боеголовку братских бурение вами великодушно ветровом взяточничества включили внучкой возле вонь востребованной всеобщие вторые выдают выложены вырасти высшую галеристам гетероструктурами глухотой горкома грамм грызет дачка действенными деревни диафрагма дневник дозированной дополнений дотягивала духовно-нравственному есенинском живое заболевание загадочных зазвучало закрывают заморозил заполняя засматривались захотелось земледелия зонтик избранники измерял имитационная

In [9]:
from spacy.lang.ru.lemmatizer import RussianLemmatizer
import ru2.lemmatizer
lemmatizer1 = RussianLemmatizer()
lemmatizer2 = ru2.lemmatizer.RussianLemmatizer()
display(lemmatizer1(u'электричеством', 'NOUN'))
display(lemmatizer1(u'электричеством', 'X'))
display(lemmatizer2(u'электричеством', 'NOUN'))
display(lemmatizer2(u'электричеством', 'X'))

['электричество']

['электричеством']

Set lemma for selected POS tag: электричеством NOUN None


['электричество']

Set lemma for selected POS tag: электричеством X None


['электричеством']

In [10]:
inspect_object(nlp_ru.vocab[100])

.cluster = 0
.flags = 8582
.has_vector = False
.lang_ = ru
.lower_ = x
.norm_ = x
.orth_ = X
.prefix_ = X
.prob = -20.0
.rank = 0
.sentiment = 0.0
.shape_ = X
.suffix_ = X
.text = X
.vocab = <spacy.vocab.Vocab object at 0x7f442e102548>
